# Analyzing Algorithmic Complexity

This notebook serves to find the complexity of each algorithm explored in the general case.

## Brute-Force

The brute-force approach is the most basic approach to solving the TSP, but also one of the slowest. It tries each path and finds which one is the fastest.

Our solution, as implemented in `brute_force.py`, runs in $O(n!)$ time. We will first prove that `_brute_force_rec` runs in $O(n!)$ time, then show that `brute_force` does nothing to increase the time complexity.

Let $P(n_p)$ be the proposition that `_brute_force_rec` runs in $O(n_p!)$ time, where $n_p$ is the number of non-0 nodes in `graph` (i.e. the number of nodes to consider) minus the length of `path`. We can see that this is true for $n_p=1$, as it runs in constant time (i.e. no loops are encountered). This is the case as, if $n_p=1$, the function will immediately enter the first `if` statement and then return, running in constant time. Constant time is $O(1)$, which is equal to $O(n_p!)$ when $n_p=1$.

We will now show that if $P(k)$ is true for some $k \ge 1$, then $P(k+1)$ is also true. Since $k+1 \ge 2$, we know that the function will bypass the first `if` statement and enter the loop. This loops repeats `len(graph) - 1` times. However, there are two options - `len(path)` of the loops run in constant time (as the only thing it does is check if `node` is in `path` - a constant-time operation since `path` is a set - and then move onto the next loop), while the remaining `k+1` loops are more complicated. Here's what it does line-by-line:

1. Check if `node` is in `path`. As `path` is a set, this takes constant time.
2. Add `node` to `path`. This also takes constant time.
3. Recurse. In this new call, as `path` is now one element bigger, $n_p=k$. By our inductive hypothesis, we know that this runs in $O(k!)$ time.
4. Remove `node` from `path`. This takes constant time.
5. Create `act_dist` as the sum of `dist` and the distance from `last_node` to `node`. This lookup and addition takes constant time.
6. Check if this produced a shorter path. Each comparison (along with updating `shortest_dist` and `shortest_path`) takes constant time.

Since we only care about the overall behavior in terms of $k+1$, we can ignore any constant-time operations, as they are negligible. This means each loop where `node` is not in `path` runs in $O(k!)$ time.

There are `len(path)` loops that run in constant time and $k+1$ loops that run in $k!$ time. This means, in total, the runtime of `brute_force` for $k+1\ge2$ is `len(path)`$+(k+1)k!$. As we only care about the long-term behavior, `len(path)` is effectively zero, leaving the runtime to be $0+(k+1)k!$. This simplifies to $O((k+1)!)$. Therefore, we have shown that if $P(k)$ is true for some $k \ge 1$, then $P(k+1)$ is also true. Since we have also shown $P(1)$, by the principle of mathematical induction, $P(n_p)$ is true for all $n_p \ge 1$. That is, `_brute_force_rec` runs in $n_p!$ time where $n_p$ is the number of non-zero nodes in `graph` minus the number of nodes in `path`.

We will now show that `brute_force` has this same time complexity, however this time in terms of $n$, the number of non-zero nodes in `graph`. `brute_force` only has two lines. We can see that the first line, which calls `_brute_force_rec` runs in $O(n!)$ time, as it passes in an empty set. That is $n_p=$`len(graph)`$-1-0=n$. The second statement returns the distance plus a constructed list. The constructed list involves a list reversal and an effective `extend` call, both of which run in linear time. The runtime of `brute_force` is therefore $O(n!+2n)$. Since we only care about the long-term behavior, we can ignore the $2n$ as it is negligible. Therefore, the true runtime of `brute_force` is $O(n!)$

## Held-Karp

The Held-Karp algorithm attempts to optimize the brute-force approach by first solving sub-problems which eliminate the need to check a high percentage of the larger problem. The basic intuition is that if you know that A->B->C-D is faster than A->C->B->D, there is no reason to check any path beginning with A->C->B->D, since it cannot be the shortest path. It does this by saving a cache mapping a tuple containing a set of nodes and a node in that set to the shortest path that goes through all of the nodes in the set, ending at the given node. For instance, `cache[({1, 2, 3}, 2)]` gives the shortest path from 0 to 2 that goes through 1, 2, and 3. It can build this mapping up from one element sets through to n-element sets.

Our solution, as implemented in `held_karp.py`, runs in $O(n^22^n)$ time, as proved below.

Let $n$ be the number of nodes in the graph in the input to `held_karp`. We will split the code into three segments, as is done in the code: the setup, part 1, and part 2. We will calculate the runtime of each part individually, and the overall runtime of the function will be whichever is the slowest.

The setup runs in $O(n)$ time. Setting each of the variables are constant, and adding a value to the cache (in the loop) is also constant. Since the number of runs of the loop is equal to $n-1$, the setup portion runs in linear time.

Part 1 runs in $O(n^22^n)$ time. Consider one loop of the outer for loop. Let $s$ be equal to `subset_size`. From there, there are $n-1 \choose s$ different subsets of size $s$ to consider. For each of these, we must loop through it twice; once to determine the target node, `k`, and once to determine the node before the target, `m`. Since these are nested, this part runs in $O(s(s-1)f(s))$ where $f(s)$ is the runtime of the innermost loop. Note the $s-1$ is there as the loop is skipped if `m==k`. Every command in the innermost loop (checking equality, dictionary lookup, addition, setting `shortest`), runs in constant time. There, $f(s)$ is constant and the innermost two loops run in $O(s(s-1))$ time (we are keeping $s(s-1)$, as opposed to $s^2$, as it will help later). As these loops are done once for each set, one run of the outer loop runs in $O(s(s-1) {n-1 \choose s})$ time. Because this is not constant with respect to $s$, the total runtime of this section is the sum of this expression over all values of $s$. That is, the runtime of part 2 is:

$\begin{equation}
\sum_{s=2}^{n-1}s(s-1){n-1 \choose s}
\end{equation}$

Simplifying,

$\begin{align}

&\sum_{s=2}^{n-1}s(s-1){n-1 \choose s} \\
=& \sum_{s=2}^{n-1}s(s-1)\frac{(n-1)!}{s!(n-1-s)!} \\
=& \sum_{s=2}^{n-1}\frac{(n-1)!}{(s-2)!(n-1-s)!} \\
=& \sum_{s=2}^{n-1}\frac{(n-1)(n-2)(n-3)!}{(s-2)!(n-3-(s-2))!} \\
=& (n-1)(n-2)\sum_{s=2}^{n-1}{n-3 \choose s-2} \\
=& (n-1)(n-2)\sum_{t=0}^{n-3}{n-3 \choose t} \\
=& (n-1)(n-2)2^{n-3} \\

\end{align}$

This means part 1 runs in approximately $O(n^22^n)$ time.

Part 2 runs in linear time. The setup before the loop takes constant time, as it is simply setting variables. Each line within the loop (dictionary lookup, addition, testing equality, setting variables) also each take constant time. The loop runs a total of $n-1$ times, so the whole section runs in linear time.

Since these three sections happen in succession, the total runtime is the sum of them. Since we only care about long-term behavior, this runtime can be approximated by the term that dominates it: $n^22^n$. Therefore, the Held-Karp algorithm has a time complexity of $O(n^22^n)$.

## Nearest Neighbor

The nearest neighbor algorithm is a fast and simple algorithm to find an *approximation* of the shortest path. This approximation is generally very bad, however, nearest neighbor (also known as "greedy") is often used as part of a more complex approximation algorithm, and so we will analyze the raw form here.

Our solution, as implemented in `nearest_neighbor.py`, runs in $O(n^2)$ time.

Let $n$ be the number of nodes in the graph in the input to `tsp_nearest_neighbor`. The bulk of the processing happens within the one loop - everything else (setting variables, appending to lists and sets, addition) takes constant time. The loop runs a total of $n-1$ times, as the size of `seen` increases by one each loop, starting from a size of 1, and it goes until it has every node in it. Everything in the loop except finding the nearest city happens in constant time. Finding the nearest city happens in linear time - for each node, `find_nearest_city` checks if it's in the list of seen nodes and finds its distance from the current node (both constant time operations) and then finds the shortest. Therefore, each loop runs in linear time. Since the number of loops is on the order of $n$, the total runtime of all the loops is $O(n^2)$. Since the rest of the function runs in constant time, `tsp_nearest_neighbor` runs in $O(n^2)$ time.